<a href="https://colab.research.google.com/github/student64-ahmadi/physic_quant/blob/main/notebook664ddc6e19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install gwpy h5py numpy matplotlib scipy requests
!python3 gw_analysis.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.9 MB/s eta 0:00:00
  Created wheel for ligo-segments: filename=ligo_segments-1.4.0-cp311-cp311-linux_x86_64.whl size=102030 sha256=7c43db5535decd9ef44917f337f994d3ef5e16c78dedbf8c4e024ec9876bc8b1
  Stored in directory: /root/.cache/pip/wheels/02/2b/93/c2ee2b6f4ea9572ee4f5ade0115c4bd0887be6228328740dae
Successfully built ligo-segments
python3: can't open file '/content/gw_analysis.py': [Errno 2] No such file or directory


In [7]:
pip install pandas h5py

In [13]:
import numpy as np
import h5py
import matplotlib
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import kstest, norm, chi2
from scipy.optimize import curve_fit
import warnings
import os

warnings.filterwarnings('ignore', category=RuntimeWarning)

# Définition des chemins des fichiers HDF5
HDF5_FILES = [
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256673280-4096.hdf5"
]

class GravitationalWaveAnalyzer:
    def __init__(self):
        self.pdf_pages = PdfPages('GW_Analysis_Report.pdf')
        self.results = {
            'valid_segments': [],
            'ks_test': [],
            'chi2_test': []
        }

    def _save_plot(self, fig):
        self.pdf_pages.savefig(fig)
        plt.close(fig)

    def _validate_data(self, data):
        if np.isnan(data).any() or np.isinf(data).any():
            data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
        if np.all(data == 0) or len(data) == 0:
            return None
        return data

    def _statistical_tests(self, data):
        ks_stat, ks_p = kstest(data, norm.cdf)
        self.results['ks_test'].append((ks_stat, ks_p))

        hist, bins = np.histogram(data, bins=50, density=True)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        expected = norm.pdf(bin_centers)
        chi_sq = np.sum((hist - expected) ** 2 / expected)
        p_value = 1 - chi2.cdf(chi_sq, len(bins) - 1)
        self.results['chi2_test'].append((chi_sq, p_value))

    def process_file(self, file_path):
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return False

        try:
            with h5py.File(file_path, 'r') as hdf:
                strain = hdf['strain/Strain'][:]
                strain = self._validate_data(strain)
                if strain is None:
                    return False

                self._statistical_tests(strain)
                self.results['valid_segments'].append(file_path)
                return True

        except Exception as e:
            print(f"Processing failed for {file_path}: {str(e)}")
            return False

    def generate_report(self):
        fig = plt.figure(figsize=(12, 8))
        plt.axis('off')
        text = [
            "Rapport d'Analyse GWOSC",
            f"Segments valides: {len(self.results['valid_segments'])}/{len(HDF5_FILES)}",
            f"KS-test moyen: {np.mean([x[0] for x in self.results['ks_test']]):.3f} ± {np.std([x[0] for x in self.results['ks_test']]):.3f}",
            f"χ² moyen: {np.mean([x[0] for x in self.results['chi2_test']]):.3f}"
        ]
        plt.text(0.1, 0.9, '\n'.join(text), fontsize=12)
        self._save_plot(fig)
        self.pdf_pages.close()

if __name__ == "__main__":
    analyzer = GravitationalWaveAnalyzer()

    for file_path in HDF5_FILES:
        print(f"Processing {file_path}...")
        analyzer.process_file(file_path)

    analyzer.generate_report()
    print("Analysis complete. Report saved to GW_Analysis_Report.pdf")


Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256673280-4096.hdf5...
Analysis complete. Report saved to GW_Analysis_Report.pdf


In [14]:
import numpy as np
import h5py
import matplotlib
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import kstest, norm, chi2
from scipy.optimize import curve_fit
import warnings
import os

warnings.filterwarnings('ignore', category=RuntimeWarning)

# Définition des chemins des fichiers HDF5
HDF5_FILES = [
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256673280-4096.hdf5"
]

class GravitationalWaveAnalyzer:
    def __init__(self):
        self.pdf_pages = PdfPages('GW_Analysis_Report.pdf')
        self.results = {
            'valid_segments': [],
            'ks_test': [],
            'chi2_test': []
        }

    def _save_plot(self, fig):
        self.pdf_pages.savefig(fig)
        plt.close(fig)

    def _validate_data(self, data):
        if np.isnan(data).any() or np.isinf(data).any():
            data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
        if np.all(data == 0) or len(data) == 0:
            return None
        return data

    def _statistical_tests(self, data):
        ks_stat, ks_p = kstest(data, norm.cdf)
        self.results['ks_test'].append((ks_stat, ks_p))

        hist, bins = np.histogram(data, bins=50, density=True)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        expected = norm.pdf(bin_centers)
        chi_sq = np.sum((hist - expected) ** 2 / expected)
        p_value = 1 - chi2.cdf(chi_sq, len(bins) - 1)
        self.results['chi2_test'].append((chi_sq, p_value))

    def process_file(self, file_path):
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return False

        try:
            with h5py.File(file_path, 'r') as hdf:
                strain = hdf['strain/Strain'][:]
                strain = self._validate_data(strain)
                if strain is None:
                    return False

                self._statistical_tests(strain)
                self.results['valid_segments'].append(file_path)

                # Ajout de visualisations
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.hist(strain, bins=100, density=True, alpha=0.6, color='b', label='Données')
                x = np.linspace(min(strain), max(strain), 100)
                ax.plot(x, norm.pdf(x, np.mean(strain), np.std(strain)), 'r-', label='Distribution normale')
                ax.set_title(f"Distribution des valeurs de bruit - {os.path.basename(file_path)}")
                ax.set_xlabel("Amplitude du bruit")
                ax.set_ylabel("Densité")
                ax.legend()
                self._save_plot(fig)

                # Analyse spectrale
                freq = np.fft.rfftfreq(len(strain), 1/4096)
                psd = np.abs(np.fft.rfft(strain))**2
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.loglog(freq, psd, 'b')
                ax.set_title(f"Spectre de puissance - {os.path.basename(file_path)}")
                ax.set_xlabel("Fréquence (Hz)")
                ax.set_ylabel("Densité spectrale de puissance")
                self._save_plot(fig)

                return True

        except Exception as e:
            print(f"Processing failed for {file_path}: {str(e)}")
            return False

    def generate_report(self):
        fig = plt.figure(figsize=(12, 8))
        plt.axis('off')
        text = [
            "Rapport d'Analyse GWOSC",
            f"Segments valides: {len(self.results['valid_segments'])}/{len(HDF5_FILES)}",
            f"KS-test moyen: {np.mean([x[0] for x in self.results['ks_test']]):.3f} ± {np.std([x[0] for x in self.results['ks_test']]):.3f}",
            f"χ² moyen: {np.mean([x[0] for x in self.results['chi2_test']]):.3f}"
        ]
        plt.text(0.1, 0.9, '\n'.join(text), fontsize=12)
        self._save_plot(fig)
        self.pdf_pages.close()

if __name__ == "__main__":
    analyzer = GravitationalWaveAnalyzer()

    for file_path in HDF5_FILES:
        print(f"Processing {file_path}...")
        analyzer.process_file(file_path)

    analyzer.generate_report()
    print("Analysis complete. Report saved to GW_Analysis_Report.pdf")


Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256673280-4096.hdf5...
Analysis complete. Report saved to GW_Analysis_Report.pdf


In [15]:
import numpy as np
import h5py
import matplotlib
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import kstest, norm, chi2
from scipy.optimize import curve_fit
import warnings
import os

warnings.filterwarnings('ignore', category=RuntimeWarning)

# Définition des chemins des fichiers HDF5
HDF5_FILES = [
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256673280-4096.hdf5"
]

class GravitationalWaveAnalyzer:
    def __init__(self):
        self.pdf_pages = PdfPages('GW_Analysis_Report.pdf')
        self.results = {
            'valid_segments': [],
            'ks_test': [],
            'chi2_test': []
        }

    def _save_plot(self, fig):
        self.pdf_pages.savefig(fig)
        plt.close(fig)

    def _validate_data(self, data):
        if np.isnan(data).any() or np.isinf(data).any():
            data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
        if np.all(data == 0) or len(data) == 0:
            return None
        return data

    def _statistical_tests(self, data):
        ks_stat, ks_p = kstest(data, norm.cdf)
        self.results['ks_test'].append((ks_stat, ks_p))

        hist, bins = np.histogram(data, bins=50, density=True)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        expected = norm.pdf(bin_centers)
        chi_sq = np.sum((hist - expected) ** 2 / expected)
        p_value = 1 - chi2.cdf(chi_sq, len(bins) - 1)
        self.results['chi2_test'].append((chi_sq, p_value))

    def generate_synthetic_signal(self, length):
        return np.random.normal(0, 1, length)

    def process_file(self, file_path):
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return False

        try:
            with h5py.File(file_path, 'r') as hdf:
                strain = hdf['strain/Strain'][:]
                strain = self._validate_data(strain)
                if strain is None:
                    return False

                self._statistical_tests(strain)
                self.results['valid_segments'].append(file_path)

                # Ajout de visualisations
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.hist(strain, bins=100, density=True, alpha=0.6, color='b', label='Données')
                x = np.linspace(min(strain), max(strain), 100)
                ax.plot(x, norm.pdf(x, np.mean(strain), np.std(strain)), 'r-', label='Distribution normale')
                ax.set_title(f"Distribution des valeurs de bruit - {os.path.basename(file_path)}")
                ax.set_xlabel("Amplitude du bruit")
                ax.set_ylabel("Densité")
                ax.legend()
                self._save_plot(fig)

                # Analyse spectrale
                freq = np.fft.rfftfreq(len(strain), 1/4096)
                psd = np.abs(np.fft.rfft(strain))**2
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.loglog(freq, psd, 'b')
                ax.set_title(f"Spectre de puissance - {os.path.basename(file_path)}")
                ax.set_xlabel("Fréquence (Hz)")
                ax.set_ylabel("Densité spectrale de puissance")
                self._save_plot(fig)

                # Comparaison avec un signal de référence
                synthetic_signal = self.generate_synthetic_signal(len(strain))
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.hist(synthetic_signal, bins=100, density=True, alpha=0.6, color='g', label='Signal de référence')
                ax.hist(strain, bins=100, density=True, alpha=0.6, color='b', label='Données réelles')
                ax.set_title("Comparaison entre le bruit observé et un signal de référence")
                ax.set_xlabel("Amplitude du signal")
                ax.set_ylabel("Densité")
                ax.legend()
                self._save_plot(fig)

                return True

        except Exception as e:
            print(f"Processing failed for {file_path}: {str(e)}")
            return False

    def generate_report(self):
        fig = plt.figure(figsize=(12, 8))
        plt.axis('off')
        text = [
            "Rapport d'Analyse GWOSC",
            f"Segments valides: {len(self.results['valid_segments'])}/{len(HDF5_FILES)}",
            f"KS-test moyen: {np.mean([x[0] for x in self.results['ks_test']]):.3f} ± {np.std([x[0] for x in self.results['ks_test']]):.3f}",
            f"χ² moyen: {np.mean([x[0] for x in self.results['chi2_test']]):.3f}"
        ]
        plt.text(0.1, 0.9, '\n'.join(text), fontsize=12)
        self._save_plot(fig)
        self.pdf_pages.close()

if __name__ == "__main__":
    analyzer = GravitationalWaveAnalyzer()

    for file_path in HDF5_FILES:
        print(f"Processing {file_path}...")
        analyzer.process_file(file_path)

    analyzer.generate_report()
    print("Analysis complete. Report saved to GW_Analysis_Report.pdf")


Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256673280-4096.hdf5...
Analysis complete. Report saved to GW_Analysis_Report.pdf


In [16]:
import numpy as np
import h5py
import matplotlib
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import kstest, norm, chi2
from scipy.optimize import curve_fit
import warnings
import os

warnings.filterwarnings('ignore', category=RuntimeWarning)

# Définition des chemins des fichiers HDF5
HDF5_FILES = [
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256673280-4096.hdf5"
]

class GravitationalWaveAnalyzer:
    def __init__(self):
        self.pdf_pages = PdfPages('GW_Analysis_Report.pdf')
        self.results = {
            'valid_segments': [],
            'ks_test': [],
            'chi2_test': []
        }

    def _save_plot(self, fig):
        self.pdf_pages.savefig(fig)
        plt.close(fig)

    def generate_synthetic_signal_extra_dimensions(self, length, freq_shift=50, decay=0.1):
        """Génère un signal synthétique simulant une propagation modifiée par des dimensions supplémentaires."""
        t = np.linspace(0, length / 4096, length)
        signal = np.sin(2 * np.pi * (100 + freq_shift) * t) * np.exp(-decay * t)
        return signal + np.random.normal(0, 0.1, length)

    def process_file(self, file_path):
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return False

        try:
            with h5py.File(file_path, 'r') as hdf:
                strain = hdf['strain/Strain'][:]
                if np.isnan(strain).any() or np.isinf(strain).any():
                    strain = np.nan_to_num(strain, nan=0.0, posinf=0.0, neginf=0.0)

                self.results['valid_segments'].append(file_path)

                # Génération d'un signal avec effets de dimensions supplémentaires
                synthetic_signal = self.generate_synthetic_signal_extra_dimensions(len(strain))

                # Comparaison des signaux
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.plot(strain[:1000], label='Données réelles', alpha=0.7)
                ax.plot(synthetic_signal[:1000], label='Signal simulé (dimensions supplémentaires)', linestyle='--')
                ax.set_title("Comparaison des signaux")
                ax.set_xlabel("Temps (échantillons)")
                ax.set_ylabel("Amplitude")
                ax.legend()
                self._save_plot(fig)

                return True

        except Exception as e:
            print(f"Processing failed for {file_path}: {str(e)}")
            return False

    def generate_report(self):
        fig = plt.figure(figsize=(12, 8))
        plt.axis('off')
        text = [
            "Rapport d'Analyse GWOSC",
            f"Segments valides: {len(self.results['valid_segments'])}/{len(HDF5_FILES)}"
        ]
        plt.text(0.1, 0.9, '\n'.join(text), fontsize=12)
        self._save_plot(fig)
        self.pdf_pages.close()

if __name__ == "__main__":
    analyzer = GravitationalWaveAnalyzer()

    for file_path in HDF5_FILES:
        print(f"Processing {file_path}...")
        analyzer.process_file(file_path)

    analyzer.generate_report()
    print("Analysis complete. Report saved to GW_Analysis_Report.pdf")


Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5...
Processing /content/H-H1_GWOSC_O3b_4KHZ_R1-1256673280-4096.hdf5...
Analysis complete. Report saved to GW_Analysis_Report.pdf


In [17]:
from gwpy.detector import ChannelList, Channel
from gwpy.timeseries import TimeSeries
from gwpy.time import tconvert
%matplotlib inline

In [23]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

--2025-03-22 00:24:16--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154615621 (147M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh.1’

Miniconda3-latest-L 100%[===================>] 147.45M   108MB/s    in 1.4s    

2025-03-22 00:24:18 (108 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh.1’ saved [154615621/154615621]

